In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cll-flowcym/CLL_162/113.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/261.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/71.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/29.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/187.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/330.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/283.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/111.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/189.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/143.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/99.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/149.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/185.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/217.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/115.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/65.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/215.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/55.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/203.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/49.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/281.fcs.txt
/kaggle/input/cll-flowcym/CLL_162/5.fcs.

In [2]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold, train_test_split
from torch import nn, optim
from tqdm import tqdm
from collections import defaultdict
import torch.nn.functional as F

# targets 
combined = pd.concat([
    pd.read_csv("https://raw.githubusercontent.com/erobl/csnn/main/metadata/CLL_test.csv"),
    pd.read_csv("https://raw.githubusercontent.com/erobl/csnn/main/metadata/CLL_train.csv"),
    pd.read_csv("https://raw.githubusercontent.com/erobl/csnn/main/metadata/CLL_val.csv")
])

combined["filename"] = combined.File
combined["label"] = combined.Label
combined.to_csv("/kaggle/working/targets.csv")

In [3]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold, train_test_split
from torch import nn, optim
from tqdm import tqdm
from collections import defaultdict

# --- Dataset ---
class FlowCytometryCSVDataset(Dataset):
    def __init__(self, csv_files, labels):
        self.csv_files = csv_files
        self.labels = labels

    def __len__(self):
        return len(self.csv_files)

    def __getitem__(self, idx):
        data = pd.read_csv(self.csv_files[idx], sep="\t")
        if "time" in data.columns:
            data = data.drop(columns=["time"])
        if "Time" in data.columns:
            data = data.drop(columns=["Time"])
        data = data.values.astype(np.float32)
        label = self.labels[idx]
        return torch.tensor(data), torch.tensor(label).long()

# --- Model ---
class AttentionMIL(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(AttentionMIL, self).__init__()
        self.embedding = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, hidden_dim),
            nn.ReLU()
        )
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(0)
        H = self.embedding(x)
        A = self.attention(H)
        A = torch.softmax(A, dim=1)
        M = torch.sum(A * H, dim=1)
        out = self.classifier(M)
        return A.squeeze(0), out.squeeze(0)

# --- Data Loading ---
def load_dataset(csv_dir, labels_csv):
    label_df = pd.read_csv(labels_csv)
    csv_files = [os.path.join(csv_dir, f) for f in label_df['filename']]
    labels = label_df['label'].values
    return csv_files, labels

# --- Training ---
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for x, y in tqdm(loader, desc="Training"):
        x, y = x[0].to(device), y.to(device)
        optimizer.zero_grad()
        _, out = model(x)
        loss = criterion(out.unsqueeze(0), y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# --- Evaluation per epoch ---
def evaluate(model, loader, device, criterion, num_classes):
    model.eval()
    total_loss = 0
    correct_per_class = defaultdict(int)
    total_per_class = defaultdict(int)

    with torch.no_grad():
        for x, y in loader:
            x, y = x[0].to(device), y.to(device)
            _, out = model(x)
            loss = criterion(out.unsqueeze(0), y)
            total_loss += loss.item()
            pred = out.argmax().item()
            label = y.item()
            total_per_class[label] += 1
            if pred == label:
                correct_per_class[label] += 1

    print("Per-class accuracy:")
    for cls in range(num_classes):
        correct = correct_per_class[cls]
        total = total_per_class[cls]
        acc = correct / total if total > 0 else 0.0
        print(f"Class {cls}: {correct}/{total} correct ({acc:.2%})")

    total_correct = sum(correct_per_class.values())
    total_instances = sum(total_per_class.values())
    avg_loss = total_loss / len(loader)
    acc = total_correct / total_instances if total_instances > 0 else 0.0
    return avg_loss, acc

# --- Main ---
def main():
    csv_dir = '/kaggle/input/cll-flowcym'
    labels_csv = '/kaggle/working/targets.csv'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    csv_files, labels = load_dataset(csv_dir, labels_csv)
    input_dim = pd.read_csv(csv_files[0], sep="\t").drop(columns=["time"], errors='ignore').shape[1]
    num_classes = len(set(labels))

    csv_trainval, csv_test, labels_trainval, labels_test = train_test_split(
        csv_files, labels, test_size=0.1, stratify=labels, random_state=42)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    patience = 20
    epochs = 200

    for fold, (train_idx, val_idx) in enumerate(skf.split(csv_trainval, labels_trainval)):
        print(f'\n--- Fold {fold + 1} ---')
        train_dataset = FlowCytometryCSVDataset(
            [csv_trainval[i] for i in train_idx],
            [labels_trainval[i] for i in train_idx]
        )
        val_dataset = FlowCytometryCSVDataset(
            [csv_trainval[i] for i in val_idx],
            [labels_trainval[i] for i in val_idx]
        )

        train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

        model = AttentionMIL(input_dim, 128, num_classes).to(device)
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        criterion = nn.CrossEntropyLoss()

        best_loss = float('inf')
        patience_counter = 0
        best_model_path = f'best_model_fold{fold+1}.pt'

        for epoch in range(epochs):
            print(f"\nEpoch {epoch+1}")
            train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
            val_loss, val_acc = evaluate(model, val_loader, device, criterion, num_classes)
            print(f'Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2%}')

            if val_loss < best_loss:
                best_loss = val_loss
                patience_counter = 0
                torch.save(model.state_dict(), best_model_path)
                print("Saved best model.")
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

        # Final test set evaluation
        model.load_state_dict(torch.load(best_model_path))
        test_dataset = FlowCytometryCSVDataset(csv_test, labels_test)
        test_loader = DataLoader(test_dataset, batch_size=1)
        print("\nTest Set Performance:")
        _, test_acc = evaluate(model, test_loader, device, criterion, num_classes)
        print(f"Test Accuracy: {test_acc:.2%}")

if __name__ == "__main__":
    main()



--- Fold 1 ---

Epoch 1


Training: 100%|██████████| 189/189 [01:48<00:00,  1.75it/s]


Per-class accuracy:
Class 0: 4/17 correct (23.53%)
Class 1: 29/31 correct (93.55%)
Train Loss: 2.0564 | Val Loss: 2.1524 | Val Acc: 68.75%
Saved best model.

Epoch 2


Training: 100%|██████████| 189/189 [00:56<00:00,  3.32it/s]


Per-class accuracy:
Class 0: 15/17 correct (88.24%)
Class 1: 22/31 correct (70.97%)
Train Loss: 1.4468 | Val Loss: 1.4703 | Val Acc: 77.08%
Saved best model.

Epoch 3


...

Per-class accuracy:
Class 0: 13/17 correct (76.47%)
Class 1: 26/30 correct (86.67%)
Train Loss: 0.2467 | Val Loss: 0.8521 | Val Acc: 82.98%

Epoch 65


Training: 100%|██████████| 190/190 [00:53<00:00,  3.52it/s]


Per-class accuracy:
Class 0: 14/17 correct (82.35%)
Class 1: 26/30 correct (86.67%)
Train Loss: 0.1184 | Val Loss: 0.6335 | Val Acc: 85.11%

Epoch 66


Training: 100%|██████████| 190/190 [00:54<00:00,  3.51it/s]


Per-class accuracy:
Class 0: 6/17 correct (35.29%)
Class 1: 28/30 correct (93.33%)
Train Loss: 0.1163 | Val Loss: 0.8353 | Val Acc: 72.34%
Early stopping at epoch 66

Test Set Performance:
Per-class accuracy:
Class 0: 9/10 correct (90.00%)
Class 1: 13/17 correct (76.47%)
Test Accuracy: 81.48%


In [4]:
# combined["filename"] = combined.File.apply(lambda x: x.split("/")[1])
# combined.to_csv("/kaggle/working/targets.csv")

In [5]:
# combined.to_csv("/kaggle/working/targets.csv")

In [6]:
    # csv_dir = '/kaggle/input/b-alldata'
    # labels_csv = '/kaggle/working/targets.csv'
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # csv_files, labels = load_dataset(csv_dir, labels_csv)
    # input_dim = pd.read_csv(csv_files[0]).shape[1]
    # num_classes = len(set(labels))

    # csv_trainval, csv_test, labels_trainval, labels_test = train_test_split(
    #     csv_files, labels, test_size=0.1, stratify=labels, random_state=42)

    # skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    # results = {}
    # results["test"] = {}
    # results["val"] = {}
    # # for fold, (train_idx, val_idx) in enumerate(skf.split(csv_trainval, labels_trainval)):
    # #     print(f'Fold {fold + 1}')
    # #     results["val"][fold+1] = {}
    # #     train_dataset = FlowCytometryCSVDataset(
    # #         [csv_trainval[i] for i in train_idx],
    # #         [labels_trainval[i] for i in train_idx]
    # #     )
    # #     val_dataset = FlowCytometryCSVDataset(
    # #         [csv_trainval[i] for i in val_idx],
    # #         [labels_trainval[i] for i in val_idx])
    # #     val_loader = DataLoader(val_dataset, batch_size=1, shuffle = False)
    # #     model = AttentionMIL(11, 128, num_classes).to("cuda")
        
        
    # #     weights_path = f"/kaggle/input/auh/tensorflow2/default/1/best_model_fold{fold+1}.pt"
    # #     model.load_state_dict(torch.load(weights_path))
    # #     model.eval()
    # #     with torch.no_grad():
    # #         # total eval 
    # #         for i, (x, y) in enumerate(val_loader):
               

    #             # patient_id = os.path.basename(val_dataset.csv_files[i])
    #             # print(f" patient: {patient_id}")
    #             # results["val"][fold+1][patient_id] = {}

    #             # # store predictions 
    #             # x, y = x[0].to(device), y.to(device)
    #             # attention, prob  = model(x)
    #             # attention, prob  = attention.cpu(), prob.cpu()
    #             # y = y.cpu()
    #             # pred = prob.argmax().item()
    #             # print(x.shape)
    #             # results["val"][fold+1][patient_id]["patient_level_probs"] = prob.numpy()[:, 1]
    #             # results["val"][fold+1][patient_id]["patient_level_pred"] = pred
    #             # results["val"][fold+1][patient_id]["patient_gt"] = y.cpu().numpy()
    #             # results["val"][fold+1][patient_id]["attention_values"] = attention.cpu().numpy()[0,:,0]
    #             # results["val"][fold+1][patient_id]["instance_features"] = x

    #             # # cell level
    #             # cell_probs = []
    #             # for cell in tqdm(x, total = len(x)): 
    #             #     cell = cell.unsqueeze(0).to(device)
    #             #     attention, prob = model(cell)
    #             #     cell_probs.extend(prob.cpu().numpy()[:, 1])
                
    #             # results["val"][fold+1][patient_id]["cell_level_probs"] = cell_probs
    #     # eval test dataset
    # test_dataset = FlowCytometryCSVDataset(csv_test, labels_test)
    # test_loader = DataLoader( test_dataset, batch_size=1, shuffle = False)

    # for fold, (train_idx, val_idx) in enumerate(skf.split(csv_trainval, labels_trainval)):
    #     print(f'Fold {fold + 1}')
    #     results["test"][fold+1] = {}
    #     model = AttentionMIL(11, 128, num_classes).to("cuda")
    #     weights_path = f"/kaggle/input/auh/tensorflow2/default/1/best_model_fold{fold+1}.pt"
    #     model.load_state_dict(torch.load(weights_path))
    #     model.eval()
    #     with torch.no_grad():
    #         # total eval 
    #         for i, (x, y) in enumerate(test_loader):

    #             patient_id = os.path.basename(test_dataset.csv_files[i])
    #             print(f" patient: {patient_id}")

    #             results["test"][fold+1][patient_id] = {}

    #             # store predictions 
    #             x, y = x[0].to(device), y.to(device)
    #             attention, prob  = model(x)
    #             attention, prob  = attention.cpu(), prob.cpu()
    #             y = y.cpu()
    #             pred = prob.argmax().item()
    #             print(x.shape)
    #             results["test"][fold+1][patient_id]["patient_level_probs"] = prob.numpy()[:, 1]
    #             results["test"][fold+1][patient_id]["patient_level_pred"] = pred
    #             results["test"][fold+1][patient_id]["patient_gt"] = y.cpu().numpy()
    #             results["test"][fold+1][patient_id]["attention_values"] = attention.cpu().numpy()[0,:,0]
    #             results["test"][fold+1][patient_id]["instance_features"] = x

    #             # cell level
    #             cell_probs = []
    #             for cell in tqdm(x, total = len(x)): 
    #                 cell = cell.unsqueeze(0).to(device)
    #                 attention, prob = model(cell)
    #                 cell_probs.extend(prob.cpu().numpy()[:, 1])
                
    #             results["test"][fold+1][patient_id]["cell_level_probs"] = cell_probs
    #     break


    # import pickle
    
    # with open("/kaggle/working/results_dict.pkl", "wb") as f:
    #     pickle.dump(results, f)




In [7]:
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import (
#     roc_auc_score, roc_curve, accuracy_score, precision_score,
#     recall_score, f1_score, confusion_matrix, average_precision_score
# )
# import pandas as pd

# # Store metrics across folds
# all_metrics = {
#     "fold": [],
#     "accuracy": [],
#     "precision": [],
#     "recall": [],
#     "f1": [],
#     "roc_auc": [],
#     "pr_auc": []
# }

# # Store FPR, TPR for AUC plotting
# roc_curves = []

# for fold in results["test"].keys():
#     y_test = []
#     y_pred_proba = []
#     y_pred = []

#     for key in results["test"][fold].keys():
#         y_pred_proba.extend(results["test"][fold][key]["patient_level_probs"])
        
#         pred_value = results["test"][fold][key]["patient_level_pred"]
#         gt_value = results["test"][fold][key]["patient_gt"]
        
#         y_test.extend(gt_value if isinstance(gt_value, (list, tuple)) else [gt_value])
#         y_pred.extend(pred_value if isinstance(pred_value, (list, tuple)) else [pred_value])

#     y_test = np.array(y_test)
#     y_pred = np.array(y_pred)
#     y_pred_proba = np.array(y_pred_proba)

#     acc = accuracy_score(y_test, y_pred)
#     prec = precision_score(y_test, y_pred)
#     rec = recall_score(y_test, y_pred)
#     f1 = f1_score(y_test, y_pred)
#     roc_auc = roc_auc_score(y_test, y_pred_proba)
#     pr_auc = average_precision_score(y_test, y_pred_proba)

#     all_metrics["fold"].append(fold)
#     all_metrics["accuracy"].append(acc)
#     all_metrics["precision"].append(prec)
#     all_metrics["recall"].append(rec)
#     all_metrics["f1"].append(f1)
#     all_metrics["roc_auc"].append(roc_auc)
#     all_metrics["pr_auc"].append(pr_auc)

#     print(f"\nFold: {fold}")
#     print(f"  Accuracy       : {acc:.4f}")
#     print(f"  Precision      : {prec:.4f}")
#     print(f"  Recall         : {rec:.4f}")
#     print(f"  F1 Score       : {f1:.4f}")
#     print(f"  ROC AUC        : {roc_auc:.4f}")
#     print(f"  PR AUC         : {pr_auc:.4f}")

#     fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
#     roc_curves.append((fold, fpr, tpr, roc_auc))

# # Plot ROC curves
# plt.figure(figsize=(10, 6))
# for fold, fpr, tpr, roc_auc in roc_curves:
#     plt.plot(fpr, tpr, label=f"{fold} (AUC = {roc_auc:.2f})")
# plt.plot([0, 1], [0, 1], "k--")
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.title("ROC Curves for 5 CV Splits")
# plt.legend(loc="lower right")
# sns.despine()
# plt.tight_layout()
# plt.show()

# # Metrics DataFrame
# df_metrics = pd.DataFrame(all_metrics)

# # Append mean row
# mean_row = df_metrics.drop(columns=["fold"]).mean().to_frame().T
# mean_row.insert(0, "fold", "mean")
# df_metrics_full = pd.concat([df_metrics, mean_row], ignore_index=True)

# # Print metrics table
# print("\nPerformance Metrics Per Fold + Average:\n")
# print(df_metrics_full.to_string(index=False))

# # Boxplot
# plt.figure(figsize=(10, 5))
# sns.boxplot(data=df_metrics.melt(id_vars="fold", var_name="metric", value_name="score"),
#             x="metric", y="score")
# plt.title("CV Fold-wise Performance Metrics")
# sns.despine()
# plt.tight_layout()
# plt.show()


In [8]:
# results["val"]

In [9]:
# /kaggle/input/auh/tensorflow2/default/1/best_model_fold4.pt
# /kaggle/input/auh/tensorflow2/default/1/best_model_fold2.pt
# /kaggle/input/auh/tensorflow2/default/1/best_model_fold5.pt
# /kaggle/input/auh/tensorflow2/default/1/best_model_fold3.pt
# /kaggle/input/auh/tensorflow2/default/1/best_model_fold1.pt